In [80]:
-- import Control.Monad.Reader

data Exp = Num Int
         | Exp :+: Exp
         | Exp :/: Exp
         | Var String
         | Let String Int Exp

type ExpEnv = [(String, Int)]

class Monad m => MonadReader r m where
  ask   :: m r
  local :: (r -> r) -> m a -> m a


In [67]:
newtype Reader r a = Reader { runReader :: r -> a }

-- muss nun erstmal alle instanzen für die kette definieren
instance Functor (Reader r) where
  fmap f (Reader m) = Reader (f . m)

instance Applicative (Reader r) where
  pure x = Reader $ const x
    --  reader braucht r -> a 
    -- e :: r (bound at :7:36)
    -- g :: r -> a (bound at :7:23)
    -- f :: r -> a -> b (bound at :7:10)
    -- mach aus e mithilfe von m2 typ a, dann nimm e und das neue und packs in m1
  Reader m1 <*> Reader m2 = Reader $ \e -> m1 e $ m2 e

instance Monad (Reader r) where
  return = pure
--   erlich gesagt, keine ahnung wie das hier funktioniert
  Reader m >>= res = Reader (\e -> runReader (res (m e)) e)


In [68]:
instance MonadReader r (Reader r) where
  ask = Reader id
  local f (Reader g) = Reader (g . f)


runReader :: Reader r a -> r -> a
runReader (Reader f) = f

In [69]:
-- evalReader :: MonadReader ExpEnv m => Exp -> m (Maybe Int)
evalReader (Num n) = return (Just n)
evalReader (Var x) = ask >>= \e1 -> return $ lookup x e1
evalReader (e1 :+: e2) = do
  res1 <- evalReader e1
  res2 <- evalReader e2
  return  $ (+) <$> res1 <*> res2
evalReader (e1 :/: e2) = do
  v1 <- evalReader e1
  v2 <- evalReader e2
  case v2 of 
    Just 0 -> return Nothing
    Just res1 -> do
        case v1 of
            Just 0 -> return $ Just 0
            Just res2 -> return $ Just $ res1 `div` res2
evalReader (Let x val e) = do
  local (\env -> (x, val) : env) (evalReader e)

In [75]:
env :: ExpEnv
env = [("x", 5), ("y", 3)]

exp :: Exp
exp = Let "x" 10 ((Var "x" :+: Var "y") :+: Var "x")

result :: Maybe Int
result = runReader (evalReader exp) env

result

Just 23